In [1]:
%load_ext autoreload
%autoreload 2
import pandas as pd
import sys
import os
# Añadir el directorio raíz del proyecto al path para importar módulos
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
if project_root not in sys.path:
    sys.path.append(project_root)

# Importaciones de los módulos del proyecto
from src.db_manager import SQLiteWrapper
from src.functions_dropbox import manage_epub_metadata
from src.data_processor import process_data
from src.functions_notion import create_books, create_annotations, create_book_pages
from src.notion_cleanup import remove_duplicate_books, clear_notion_database, clean_all_notion_databases, clear_all_book_pages
from src.config import NOTION_API_TOKEN, NOTION_BOOKS_DATABASE_ID, NOTION_ANNOTATIONS_DATABASE_ID
from notion_client import Client

pd.options.display.max_columns = None

# Carga de BBDD

In [2]:
db_path = os.path.join("..", "data", "KoboReader.sqlite") 
db = SQLiteWrapper(db_path)
db.connect()  # Establecer conexión
anotaciones_df = db.get_annotations()
libros_ereader_df = db.get_books()
# db.close()

# Dropbox

In [3]:
# Obtener metadatos de Dropbox con caché inteligente
epub_metadata = manage_epub_metadata(
    libros_ereader_df, 
    cache_path=os.path.join("..", "data", "epub_metadata.pkl"),
    folder_path='/Aplicaciones/Rakuten Kobo'
)


🔄 Obteniendo metadatos de Dropbox...
   -> Usando caché de metadatos.
   -> 231 libros nuevos detectados. Obteniendo sus metadatos de Dropbox...
   -> Caché de metadatos actualizada.
   -> Caché de metadatos actualizada.


In [4]:
# Procesar y enriquecer los datos usando la función modularizada
libros_df = process_data(anotaciones_df, libros_ereader_df, epub_metadata)


🔧 Procesando y enriqueciendo datos...
✅ Datos procesados.


# Exportacion a Notion

In [5]:
notion = Client(auth=NOTION_API_TOKEN)

In [10]:
## 🧹 Mantenimiento de Notion (Opcional)

# ⚠️ **Ejecuta estas celdas solo cuando sea necesario:**
# - **Opción 1**: Eliminar libros duplicados (mantiene la primera ocurrencia)
# remove_duplicate_books(notion, NOTION_BOOKS_DATABASE_ID)
# - **Opción 2**: Limpiar solo la base de datos de libros
# clear_notion_database(notion, NOTION_BOOKS_DATABASE_ID, "Libros", confirm=True)
# - **Opción 3**: Limpiar solo la base de datos de anotaciones
clear_notion_database(notion, NOTION_ANNOTATIONS_DATABASE_ID, "Anotaciones", confirm=False)
# - **Opción 4**: Limpiar ambas bases de datos (empezar desde cero)
# clean_all_notion_databases(notion, NOTION_BOOKS_DATABASE_ID, NOTION_ANNOTATIONS_DATABASE_ID, confirm=True)
# - **Opción 5**: Limpiar solo el contenido de las páginas de libros (mantiene libros y anotaciones)
# clear_all_book_pages(notion, NOTION_BOOKS_DATABASE_ID, confirm=True)


🔍 Obteniendo todos los registros de Anotaciones...
   📚 Total de registros encontrados: 585
🗑️  Eliminando 585 registros en paralelo...
   📚 Total de registros encontrados: 585
🗑️  Eliminando 585 registros en paralelo...


🗑️  Eliminando: 100%|██████████| 585/585 [00:39<00:00, 14.93 registros/s]

✅ Anotaciones: 585 registros eliminados, 0 fallidos.


585

In [7]:
# Configuración inicial del cliente de Notion
notion = Client(auth=NOTION_API_TOKEN)

# 1. Crear/actualizar los libros en la base de datos de libros
print("📚 Sincronizando libros...")
create_books(libros_df, notion, NOTION_BOOKS_DATABASE_ID, force_update=False)

📚 Sincronizando libros...
🔍 Cargando libros existentes de Notion...
🔍 Cargando libros existentes de Notion...
   📚 Total de libros en Notion: 198
   📚 Total de libros en Notion: 198


📚 Sincronizando libros: 100%|██████████| 197/197 [00:12<00:00, 16.10it/s]


📚 Libros procesados: 0 creados, 19 actualizados, 178 sin cambios


In [12]:
# 2. Crear solo las anotaciones nuevas
print("\n📝 Sincronizando anotaciones...")
create_annotations(anotaciones_df, notion, NOTION_ANNOTATIONS_DATABASE_ID, NOTION_BOOKS_DATABASE_ID)


📝 Sincronizando anotaciones...
🔍 Verificando anotaciones existentes en Notion...
🔍 Verificando anotaciones existentes en Notion...
📝 2153 anotaciones en Notion, 3297 únicas en Kobo, procesando 1144 nuevas
📝 2153 anotaciones en Notion, 3297 únicas en Kobo, procesando 1144 nuevas
📝 Creando 1144 anotaciones en paralelo...
📝 Creando 1144 anotaciones en paralelo...


Creando anotaciones: 100%|██████████| 1144/1144 [01:58<00:00,  9.68anotación/s]

✅ 1144 anotaciones creadas exitosamente


In [9]:
# 3. Actualizar el contenido de las páginas de los libros (solo si hay cambios)
print("\n📖 Actualizando páginas de libros...")
create_book_pages(anotaciones_df, notion, NOTION_BOOKS_DATABASE_ID, force_update=False)


📖 Actualizando páginas de libros...
🔍 Obteniendo información de libros...
📖 Procesando 35 libros en paralelo...


Procesando libros: 100%|██████████| 35/35 [06:28<00:00, 11.10s/libro]

📚 Páginas procesadas: 3 creadas, 20 actualizadas, 12 sin cambios
